In [1]:
from vpython import*
from queue import PriorityQueue

<IPython.core.display.Javascript object>

In [2]:
# class to instantiate nets for each module
class Net:
    def __init__(self, driver, instance, time, on=0):
        self.value = None
        self.instance = instance
        self.driver = driver
        self.receivers = []
        self.on = on 
        self.time = time
        
    # function to connect each net to net of another module
    def connect(self, inputs):
        if not isinstance(inputs, list):
            inputs = [inputs]
            
        for input in inputs:
            self.receivers.append(input)

    def set(self, value):
        if self.value == value:
            return 
        
        self.value = value
        
        if self.on:
            self.driver.evaluate()
            
        for c in self.receivers:
            c.set(value)

# helper class for evaluating logic for each gate
class Logic:
    def __init__(self, name):
        self.name = name

    def evaluate(self):
        return

# helper class for instantiating 3 nets for a 2 input gate 
class Gate(Logic):        
    def __init__(self, name, instance, delay):
        Logic.__init__(self, name)
        self.instance = instance
        self.A = Net(self, 0, time=0, on=1) # input 1
        self.B = Net(self, 1, time=0, on=1) # input 2
        self.C = Net(self, 2, time=delay, on=1) # output
        
# class to create a NAND gate     
class Nand(Gate):       
    def __init__(self, name, instance):
        Gate.__init__(self, name, instance, 1)
        
    def evaluate(self):
        result = not(self.A.value and self.B.value)
        self.C.set(int(result))

# class to create a NOR gate
class Nor(Gate):         
    def __init__(self, name, instance):
        Gate.__init__(self, name, instance, 1)

    def evaluate(self):
        result = not(self.A.value or self.B.value)
        self.C.set(int(result))

# class to create an inverter 
class Not(Logic):     
    def __init__(self, name, instance, delay):
        Logic.__init__(self, name)
        self.instance = instance
        self.A = Net(self, 0, time=0, on=1) # input
        self.B = Net(self, 2, time=delay, on=1) # output

    def evaluate(self):
        result = not self.A.value
        self.B.set(int(result))

# class to instantiate an event holding attributes of the net passed and overloaded comparison operators
# for a priority queue to handle
class Event:
    
    def __init__(self, net, t):
        self.instance = net 
        self.value = net.value 
        self.time = net.time + t
        self.type = net.instance
        
    def __lt__(self, other): return self.time < other.time
    
    def __le__(self, other): return self.time <= other.time
        
    def __eq__(self, other): return self.time == other.time
    
    def __gt__(self, other): return self.time > other.time
    
    def __ge__(self, other): return self.time >= other.time
        
    
# function to set input values of a module       
def set_input(gate, a, b):
    gate.A.set(a)
    gate.B.set(b)
    
def update_net(gate, input_values, index):
    a,b = input_values[index]
    set_input(gate, a, b)
    
def get_event(eq1, eq2, eq3):
    e1 = eq1.get()
    e2 = eq2.get()
    e3 = eq3.get()
    
    return e1, e2, e3

def setup_event(gate, values, time):
    event_q1 = PriorityQueue()
    event_q2 = PriorityQueue()
    event_q3 = PriorityQueue()

    update_net(gate, values, 0)

    e1 = Event(gate.A, time)
    e2 = Event(gate.B, time)
    e3 = Event(gate.C, time)

    event_q1.put(e1)
    event_q2.put(e2)
    event_q3.put(e3)
    
    return event_q1, event_q2, event_q3

def setup_Net_event(gate, input1, input2, values, time):
    event_q1 = PriorityQueue()
    event_q2 = PriorityQueue()
    event_q3 = PriorityQueue()

    update_net(input1, values, 0)
    update_net(input2, values, 0)

    e1 = Event(gate.A, time)
    e2 = Event(gate.B, time)
    e3 = Event(gate.C, time)

    event_q1.put(e1)
    event_q2.put(e2)
    event_q3.put(e3)
    
    return event_q1, event_q2, event_q3


# function to run discrete event time simulation 
def simulate_net(t, clock, setup, values, gate, g1, g2, g3):
    index = 1
    eq1, eq2, eq3 = setup
    while not (eq3.empty()):
        # get next event
        event1, event2, event3 = get_event(eq1, eq2, eq3)

        # update time
        t += event3.time - clock
        clock = t

        g1.plot(clock, event1.value)
        g2.plot(clock, event2.value)
        g3.plot(clock, event3.value)


        if index < len(values):
            # update_net(event)
            update_net(gate, values, index)
            index += 1

            next_event1 = Event(event1.instance, clock)
            next_event2 = Event(event2.instance, clock)
            next_event3 = Event(event3.instance, clock)

            # post new event
            eq1.put(next_event1)
            eq2.put(next_event2)
            eq3.put(next_event3)

            # update visualization
            g1.plot(clock, next_event1.value)
            g2.plot(clock, next_event2.value)
            g3.plot(clock, next_event3.value)

            
def simulate_net2(t, clock, setup, values, gate, gate2, g1, g2, g3):
    index = 1
    eq1, eq2, eq3 = setup
    while not (eq3.empty()):
        # get next event
        event1, event2, event3 = get_event(eq1, eq2, eq3)

        # update time
        t += event3.time - clock
        clock = t

        g1.plot(clock, event1.value)
        g2.plot(clock, event2.value)
        g3.plot(clock, event3.value)


        if index < len(values):
            # update_net(event)
            update_net(gate, values, index)
            update_net(gate2, values, index)
            index += 1

            next_event1 = Event(event1.instance, clock)
            next_event2 = Event(event2.instance, clock)
            next_event3 = Event(event3.instance, clock)

            # post new event
            eq1.put(next_event1)
            eq2.put(next_event2)
            eq3.put(next_event3)

            # update visualization
            g1.plot(clock, next_event1.value)
            g2.plot(clock, next_event2.value)
            g3.plot(clock, next_event3.value)


In [3]:
# set up Nand gate
t = 0
clock = 0

truth_values = [(1,1), (0,0), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]
nand = Nand('nand1', 0)

# Verification of NAND nets (inputs and output)

In [4]:
scene1 = canvas()

<IPython.core.display.Javascript object>

In [5]:
# define graphs and simulate values
g1 = graph(title="Input1", xtitle='time', ytitle='value', fast=False, width=800)
g2 = graph(title="Input2", xtitle='time', ytitle='value', fast=False, width=800)
g3 = graph(title="Output", xtitle='time', ytitle='value', fast=False, width=900)
net1 = gcurve(graph=g1, color=color.blue, width=4, markers=True, marker_color=color.black, label='net1')
net2 = gcurve(graph=g2, color=color.yellow, width=4, markers=True, marker_color=color.black, label='net2')
net3 = gcurve(graph=g3, color=color.red, width=4, markers=True, marker_color=color.black, label='nand1')

simulate_net(t, clock, setup_event(nand, truth_values, t), truth_values, nand, net1, net2, net3)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Output displays correct data for NAND gate where the input data is a truth table**

# Verification of NOR nets (inputs and output)

In [9]:
scene2 = canvas()

<IPython.core.display.Javascript object>

In [10]:
# define new graphs 
g4 = graph(title="Input1", xtitle='time', ytitle='value', fast=False, width=800)
g5 = graph(title="Input2", xtitle='time', ytitle='value', fast=False, width=800)
g6 = graph(title="Output", xtitle='time', ytitle='value', fast=False, width=900)
net4 = gcurve(graph=g4, color=color.blue, width=4, markers=True, marker_color=color.black, label='net1')
net5 = gcurve(graph=g5, color=color.yellow, width=4, markers=True, marker_color=color.black, label='net2')
net6 = gcurve(graph=g6, color=color.red, width=4, markers=True, marker_color=color.black, label='nor1')

# set up NOR gate for simulation
truth_table = [(0,1), (0,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]
t = 0
clock = 0
nor = Nor('nor1', 0)

# run simulation
simulate_net(t, clock, setup_event(nor, truth_table, t), truth_table, nor, net4, net5, net6)

**Output displays correct data for NOR gate where the input data is a truth table**

# Combinational NAND gate network

In [11]:
scene3 = canvas()

<IPython.core.display.Javascript object>

In [12]:
g7 = graph(title="NAND1 Output", xtitle='time', ytitle='value', fast=False, width=800)
g8 = graph(title="NAND2 Output", xtitle='time', ytitle='value', fast=False, width=800)
g9 = graph(title="Network Output", xtitle='time', ytitle='value', fast=False, width=900)
net7 = gcurve(graph=g7, color=color.blue, width=4, markers=True, marker_color=color.black, label='net1')
net8 = gcurve(graph=g8, color=color.yellow, width=4, markers=True, marker_color=color.black, label='net2')
net9 = gcurve(graph=g9, color=color.red, width=4, markers=True, marker_color=color.black, label='Nand3')

t = 0
clock = 0

nand1 = Nand('nand1', 0)
nand2 = Nand('nand2', 1)
nand3 = Nand('nand3', 2)

nand1.C.connect([nand3.A])
nand2.C.connect([nand3.B])

simulate_net2(t, clock, setup_Net_event(nand3, nand1, nand2, truth_values, t), truth_values, nand1, nand2, net7, net8, net9)

# Combinational NOR gate network 

In [13]:
scene4 = canvas()

<IPython.core.display.Javascript object>

In [14]:
g10 = graph(title="NOR1 Output", xtitle='time', ytitle='value', fast=False, width=800)
g11 = graph(title="NOR2 Output", xtitle='time', ytitle='value', fast=False, width=800)
g12 = graph(title="Network Output", xtitle='time', ytitle='value', fast=False, width=900)
net10 = gcurve(graph=g10, color=color.blue, width=4, markers=True, marker_color=color.black, label='net1')
net11 = gcurve(graph=g11, color=color.yellow, width=4, markers=True, marker_color=color.black, label='net2')
net12 = gcurve(graph=g12, color=color.red, width=4, markers=True, marker_color=color.black, label='NOR3')

t = 0
clock = 0

nor1 = Nor('nor1', 0)
nor2 = Nor('nor2', 1)
nor3 = Nor('nor3', 2)

nor1.C.connect([nor3.A])
nor2.C.connect([nor3.B])

simulate_net2(t, clock, setup_Net_event(nor3, nor1, nor2, truth_values, t), truth_values, nor1, nor2, net10, net11, net12)

# NAND-Gate D Latch

In [15]:
# set up D latch network
clk = 0
sysClk = 0

clock_values = [0,1,1,0,1,1,0,1,1,0,0]
data_values =  [0,0,1,0,0,1,0,0,1,1,0]

nand1 = Nand('nand1_S', 0)
nand2 = Nand('nand2_R', 1)
nand3 = Nand('nand3_Q', 2)
nand4 = Nand('nand4_Qb', 3)
not1 = Not('not1', 0, 1)

nand1.B.connect([nand2.A])
nand1.C.connect([nand3.A])
nand1.A.connect([not1.A])
not1.B.connect([nand2.B])
nand4.C.connect([nand3.B])
nand2.C.connect([nand4.B])
nand3.C.connect([nand4.A])

nand1.A.set(data_values[0])
nand1.B.set(clock_values[0])

eq_Q = PriorityQueue()
eq_Qb = PriorityQueue()
eq_D = PriorityQueue()
eq_clk = PriorityQueue()

ev_D = Event(nand1.A, clk)
ev_clk = Event(nand2.A, clk)
ev_Q = Event(nand3.C, clk)
ev_Qb = Event(nand4.C, clk)

eq_Q.put(ev_Q)
eq_Qb.put(ev_Qb)
eq_D.put(ev_D)
eq_clk.put(ev_clk)

gr1 = graph(title="Q", xtitle='time', ytitle='value', fast=False, width=800)
gr2 = graph(title="Qb", xtitle='time', ytitle='value', fast=False, width=800)
gr3 = graph(title="D", xtitle='time', ytitle='value', fast=False, width=800)
gr4 = graph(title="Clk", xtitle='time', ytitle='value', fast=False, width=875)

Qb_g = gcurve(graph=gr2, color=color.yellow, width=4, markers=True, marker_color=color.black, label='Qb')
Q_g = gcurve(graph=gr1, color=color.blue, width=4, markers=True, marker_color=color.black, label='Q')
D_g = gcurve(graph=gr3, color=color.cyan, width=4, markers=True, marker_color=color.black, label='Data')
clk_g = gcurve(graph=gr4, color=color.red, width=4, markers=True, marker_color=color.black, label='Clk')

In [16]:
scene5 = canvas()

<IPython.core.display.Javascript object>

In [17]:
i = 1
while not eq_Q.empty():
    # get next event
    e1 = eq_Q.get()
    e2 = eq_Qb.get()
    e3 = eq_D.get()
    e4 = eq_clk.get()
    
    # update time
    clk += e1.time - sysClk
    sysClk = clk
    
    Q_g.plot(sysClk, e1.value)
    Qb_g.plot(sysClk, e2.value)
    D_g.plot(sysClk, e3.value)
    clk_g.plot(sysClk, e4.value)
    
    if i < len(clock_values):
        # update_net(event)
        nand1.A.set(data_values[i])
        nand1.B.set(clock_values[i])
        i += 1
        
        next_e1 = Event(e1.instance, sysClk)
        next_e2 = Event(e2.instance, sysClk)
        next_e3 = Event(e3.instance, sysClk)
        next_e4 = Event(e4.instance, sysClk)
        
        # post new event
        eq_Q.put(next_e1)
        eq_Qb.put(next_e2)
        eq_D.put(next_e3)
        eq_clk.put(next_e4)
        
        # update visualization
        Q_g.plot(sysClk, next_e1.value)
        Qb_g.plot(sysClk, next_e2.value)
        D_g.plot(sysClk, next_e3.value)
        clk_g.plot(sysClk, next_e4.value)

# Verification 

Q output holds data from D only when clock is on.  When clock is off, Q retains data from D. Qb outputs complement of Q.